<a href="https://colab.research.google.com/github/budoorhza/PROJECT363/blob/main/bhbhb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
zip_path = '/content/drive/MyDrive/PROJECT363/BrainTumorDataset.zip'
import shutil
extract_path = '/content/drive/MyDrive/PROJECT363'
shutil.unpack_archive(zip_path, extract_path)

In [3]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [4]:
def load_data(main_dir, img_size=(128, 128)):
    images, masks = [], []
    classes = {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}  # تصنيفات الفئات
    for category in classes.keys():
        category_path = os.path.join(main_dir, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)

            # قراءة الصورة
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))  # تأكد من الحجم (128, 128)
            images.append(img)

            # إنشاء القناع
            mask = classes[category]
            masks.append(mask)

    images = np.array(images) / 255.0  # تطبيع الصور
    images = np.expand_dims(images, axis=-1)  # إضافة بُعد القناة
    masks = np.array(masks)  # تحويل الأقنعة إلى مصفوفة
    return images, masks

# مسارات المجلدات
training_dir = '/content/drive/MyDrive/PROJECT363/Training'
testing_dir = '/content/drive/MyDrive/PROJECT363/Testing'


# تحميل بيانات التدريب والاختبار
X_train, y_train = load_data(training_dir)
X_test, y_test = load_data(testing_dir)

print("Original y_train shape:", y_train.shape)
print("Original y_test shape:", y_test.shape)

# دالة لتحويل التصنيفات إلى أقنعة بكسلية
def create_pixel_masks(y, img_size=(128, 128), num_classes=4):
    masks = []
    for label in y:
        mask = np.zeros((img_size[0], img_size[1], num_classes))
        mask[..., int(label)] = 1  # تحويل الفئة إلى One-Hot
        masks.append(mask)
    return np.array(masks)

# إنشاء أقنعة بكسلية
y_train = create_pixel_masks(y_train, img_size=(128, 128), num_classes=4)
y_test = create_pixel_masks(y_test, img_size=(128, 128), num_classes=4)

# التأكد من الأشكال الجديدة
print("y_train shape after conversion:", y_train.shape)
print("y_test shape after conversion:", y_test.shape)

Original y_train shape: (5712,)
Original y_test shape: (1311,)
y_train shape after conversion: (5712, 128, 128, 4)
y_test shape after conversion: (1311, 128, 128, 4)


In [5]:
print("Original y_train shape:", y_train.shape)

Original y_train shape: (5712, 128, 128, 4)


In [6]:
# تحويل y_train و y_test إلى أقنعة لكل بكسل
def create_pixel_masks(y, img_size=(128, 128), num_classes=4):
    masks = []
    for label in y:
        # إنشاء قناع لكل تصنيف بحجم الصورة
        mask = np.full((img_size[0], img_size[1], num_classes), label.argmax(), dtype=np.float32)
        masks.append(mask)
    return np.array(masks)

# إنشاء أقنعة بكسلية
y_train = create_pixel_masks(y_train, img_size=(128, 128), num_classes=4)
y_test = create_pixel_masks(y_test, img_size=(128, 128), num_classes=4)

# التأكد من الأشكال
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (5712, 128, 128, 1)
y_train shape: (5712, 128, 128, 4)
X_test shape: (1311, 128, 128, 1)
y_test shape: (1311, 128, 128, 4)


In [7]:
def unet_model(input_size=(128, 128, 1), num_classes=4):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    # Decoder
    up4 = UpSampling2D(size=(2, 2))(conv3)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up5)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv5)

    model = Model(inputs, outputs)
    #model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = unet_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 128)         │          36,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 64, 64, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 128)         │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 128, 128, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 128, 128, 64)        │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 128, 128, 4)         │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 701,572 (2.68 MB)

 Trainable params: 701,572 (2.68 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
print("X_train min, max:", X_train.min(), X_train.max())
print("y_train min, max:", y_train.min(), y_train.max())

X_train min, max: 0.0 1.0
y_train min, max: 0.0 3.0


In [9]:
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=16
)

Epoch 1/5
  4/322 ━━━━━━━━━━━━━━━━━━━━ 44:24 8s/step - accuracy: 0.0096 - loss: 8.2973

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9b160f37d5c8>", line 1, in <cell line: 1>
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py", line 320, in fit
    logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py", line 833, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function

TypeError: object of type 'NoneType' has no len()